# Vestwell Python Screener

If you have any questions or feel you are making assumptions, please record them in this notebook or in comments if you'd rather work in a `.py` file.  If you get stuck, try to explain in words how you would complete the task(s).

### Background

Vestwell provides a wide variety of investment choices to its users.  Participants in a retirement plan can choose between a pre-determined set of funds or they can choose their own custom set of funds from a list of choices.  Advisors can create their own models with a custom set of funds in which participants can choose to invest.  As a result, there are thousands of unique models on the Vestwell platform.  

One of Vestwell's partners has the same list of models in its database.  This partner will maintain an up-to-date list of funds for each model in their database.  For example, when a fund closes and is replaced by a new one, Vestwell's partner will update the model with the new fund in their database, but not in Vestwell's.  For this reason, Vestwell's database and our partner's database will get out of sync over time.  Unless, that is, you can build python script to reconcile the two databases.  We're rooting for you!

### The Data

Here's a high-level overview of the data.  We'll get into more details below as we dig in.

**Vestwell Data**

Each `program_id` has many `model_id`s.  Each `model_id` has many `symbol`s.

* model.csv:  Associations of programs to models.
* model_prop.csv:  Association of models to symbols.

**Partner Data**

Each `PLANID` has many `FUNDID`s.

* partner.csv:  Association of `PLANID` to `FUNDID` and `PLANINVCLOSEDATE`.  

**Some extra notes**

* The `FUNDID` in our partner's data is equivalent to `symbol` in Vestwell's data.  These are also referred to as "funds".
* The `PLANID` in our partner's data has information that is equivalent to the `program_id` in Vestwell's database (more details below in Step 2).
* The `PLANINVCLOSEDATE` in our partner's database is the date when a fund was closed.  If there isn't a date, then the fund has not been closed.
* Sometimes our partner has funds called either "Medicham" or "Electrike" which we ignore.

### Goal
The goal of this exercise is to compare Vestwell's data with our partner's data.  We want to figure out if Vestwell's model data is the same as our Partner's model data.  We consider our partner's database the source of truth since their database will remain updated if there are any changes to funds.  Here's specifically what we are asking:

1.  Do the list of funds for each `program_id` in Vestwell's database match the list of funds in our partner's database?  If there are any mismatches, what funds are missing from each database?   

For example, if Vestwell's database has funds A, B and C for a `program_id` and our partner's database has funds B, C, and D for the same `program_id` we would report that fund A is missing from our partner's database and that fund D is missing from our database.

2.  Are there any funds in Vestwell's database that have closed?  If so, what are they for each `program_id`?

For example, if our database has funds D, E, and F for a `program_id` and partner's database shows that fund D closed on 11/1/2019, then we would report that fund D has closed for that `program_id`.

Ideally, the output is in a form that can be passed to a Business Analyst to take action on.  For example, the output could look something like this:

| program_id | fund_missing_at_vw | fund_missing_at_partner | fund_closed |
|------------|---------------|--------------------|--------|
| 1          | None          | None               | None   |
| 2          | D, Z             | A                  | F   |
| 3          | F             | None               | F      |

## Table Schema


![Schema](data/Vestwell_challange.png)

## Step 0
Import any packages you'll need

In [352]:
import pandas as pd
import numpy as np
import math

## Step 1
Import `partner.csv`, `model.csv`, and `model_prop.csv`.

In [353]:
model = pd.read_csv('data/model_abridged.csv')
model.head()

,model_id,program_id
0,28,3
1,34,4
2,42,4
3,24,3
4,64,8


In [354]:
model_prop = pd.read_csv('data/model_props_abridged.csv')
model_prop.head()


,model_props_id,model_id,symbol
0,541,80,Bulbasaur
1,542,80,Ivysaur
2,543,80,Venusaur
3,544,80,VenusaurMega Venusaur
4,545,80,Charmander


In [355]:
partner = pd.read_csv('data/partner_abridged.csv')
partner.head()

,PLANID,PLANINVCLOSEDATE,FUNDID
0,VW0008000039,active,Medicham
1,VW0008000039,active,Arcanine
2,VW0008000039,11/01/2018,Clefairy
3,VW0008000039,active,Zubat
4,VW0008000039,active,Nidoking


In [356]:
partner.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 445 entries, 0 to 444
Data columns (total 3 columns):
PLANID              445 non-null object
PLANINVCLOSEDATE    445 non-null object
FUNDID              445 non-null object
dtypes: object(3)
memory usage: 10.5+ KB


## Find Unique Columns in each table

### in Model table model_id is unique

In [357]:
#model.model_id looks like a unique key
model.model_id.value_counts()

336    1
74     1
83     1
82     1
81     1
80     1
335    1
347    1
333    1
332    1
75     1
73     1
62     1
72     1
71     1
326    1
69     1
68     1
67     1
66     1
65     1
64     1
84     1
85     1
86     1
87     1
106    1
105    1
104    1
103    1
      ..
374    1
378    1
379    1
113    1
112    1
111    1
223    1
352    1
100    1
151    1
328    1
19     1
329    1
278    1
334    1
349    1
18     1
163    1
162    1
161    1
160    1
159    1
158    1
157    1
156    1
155    1
154    1
153    1
152    1
282    1
Name: model_id, Length: 187, dtype: int64

### in model_prop table model_props_id is unique

In [358]:
#model.model_id looks like a unique key
model_prop.model_props_id.value_counts()

2525    1
1382    1
354     1
353     1
352     1
351     1
350     1
349     1
348     1
347     1
346     1
345     1
344     1
343     1
342     1
341     1
340     1
339     1
338     1
337     1
336     1
1381    1
1383    1
334     1
1384    1
1629    1
1884    1
1873    1
2738    1
1399    1
       ..
632     1
631     1
630     1
629     1
628     1
627     1
626     1
645     1
646     1
647     1
1659    1
2714    1
1630    1
2712    1
2711    1
2530    1
2709    1
2708    1
2707    1
2705    1
2542    1
1626    1
2703    1
2702    1
2701    1
2700    1
2699    1
2698    1
2697    1
1615    1
Name: model_props_id, Length: 729, dtype: int64

### Partner table does NOT have any unique columns

In [359]:
partner.FUNDID.value_counts()

Electrike                    18
Medicham                     12
CharizardMega Charizard X     9
Charmander                    9
CharizardMega Charizard Y     6
Bulbasaur                     6
Gastly                        6
Charmeleon                    6
BlastoiseMega Blastoise       6
Farfetch'd                    5
Magnemite                     5
Slowbro                       5
Blastoise                     5
VenusaurMega Venusaur         5
Rapidash                      5
Magikarp                      4
SlowbroMega Slowbro           4
Venusaur                      4
Growlithe                     4
Ponyta                        4
Doduo                         4
Slowpoke                      4
Squirtle                      4
Magneton                      4
Charizard                     4
Haunter                       4
Xatu                          3
Nidorino                      3
Raichu                        3
Geodude                       3
                             ..
Elekid  

In [360]:
partner.PLANID.value_counts()

VW0027000228    39
VW0009000188    36
VW0015000141    35
VW0008000039    31
VW0012000114    28
VW0030000255    26
VW0021000178    25
VW0013000121    25
VW0014000136    25
VW0020000173    24
VW0017000143    24
VW0029000243    22
VW0019000168    20
VW0016000137    19
VWPALL000076    18
VW0028000216    17
VW0024000187    16
VW0010000135    15
Name: PLANID, dtype: int64

## Step 2 - working with the `partner.csv` data
Extract the `program_id` from the `PLANID` column in the `partner` dataframe.  The `program_id` is the first four characters in `PLANID` after "VW".  It's usually an integer.  If instead of digits, those characters are equal to "PALL" then the `program_id` = 1.  Drop any other rows remaining that do not have four digits in the first four characters after "VW" in the `PLANID` column.

In [361]:
def get_plan(plan_id):
    if plan_id[2:6]=='PALL':
        return 1
    #need to remove leading zeros 00
    #try except will only return the int on 2:6 if its a number
    try:   
        return int(plan_id[2:6])
    #otherwise return nan to be dropped
    except:
        return np.nan

In [362]:
partner.head(10)

,PLANID,PLANINVCLOSEDATE,FUNDID
0,VW0008000039,active,Medicham
1,VW0008000039,active,Arcanine
2,VW0008000039,11/01/2018,Clefairy
3,VW0008000039,active,Zubat
4,VW0008000039,active,Nidoking
5,VW0008000039,active,Jigglypuff
6,VW0008000039,active,CharizardMega Charizard X
7,VW0008000039,active,Electrike
8,VW0008000039,active,Growlithe
9,VW0008000039,active,Fearow


In [363]:
partner['partner_program_id'] = 0

In [364]:
partner.partner_program_id = partner.PLANID.apply(lambda x: get_plan(x))

In [365]:
#need to drop the NAN columns
partner.dropna(inplace=True)

In [366]:
partner.shape

(445, 4)

In [367]:
partner.head(10)

,PLANID,PLANINVCLOSEDATE,FUNDID,partner_program_id
0,VW0008000039,active,Medicham,8
1,VW0008000039,active,Arcanine,8
2,VW0008000039,11/01/2018,Clefairy,8
3,VW0008000039,active,Zubat,8
4,VW0008000039,active,Nidoking,8
5,VW0008000039,active,Jigglypuff,8
6,VW0008000039,active,CharizardMega Charizard X,8
7,VW0008000039,active,Electrike,8
8,VW0008000039,active,Growlithe,8
9,VW0008000039,active,Fearow,8


## Step 3
Check if the funds match for each `program_id`.  In `partner.csv` the funds are in the `FUNDID` column and for `model_prop.csv` the funds are in the `symbol` column.  If there are any mismatches, return a list of which funds are missing from each database for each `program_id`.

### Add model.program_id to model_prop table

In [368]:
model_prop['program_id']= np.nan

In [369]:
#function used to populate the program_id column
def get_program_id(dfx, model_id):
#     df.loc[5].at['B']
#     dfx.loc[model_id]
    return dfx.set_index('model_id').at[model_id,'program_id']

In [370]:
model_prop.program_id = model_prop.model_id.apply(lambda x: get_program_id(model, x))

In [371]:
#delete later
model_prop[model_prop.program_id==8]

,model_props_id,model_id,symbol,program_id
33,284,53,Raticate,8
34,285,54,Spearow,8
35,286,55,Fearow,8
36,287,56,Ekans,8
37,288,57,Arbok,8
38,289,58,Pikachu,8
39,290,59,Raichu,8
40,291,60,Sandshrew,8
41,292,61,Sandslash,8
42,293,62,Nidorina,8


In [372]:
#delete later
partner.head(40)

,PLANID,PLANINVCLOSEDATE,FUNDID,partner_program_id
0,VW0008000039,active,Medicham,8
1,VW0008000039,active,Arcanine,8
2,VW0008000039,11/01/2018,Clefairy,8
3,VW0008000039,active,Zubat,8
4,VW0008000039,active,Nidoking,8
5,VW0008000039,active,Jigglypuff,8
6,VW0008000039,active,CharizardMega Charizard X,8
7,VW0008000039,active,Electrike,8
8,VW0008000039,active,Growlithe,8
9,VW0008000039,active,Fearow,8


In [373]:
combined_df = pd.merge(partner, model_prop,  how='outer', left_on=['FUNDID', 'partner_program_id'], 
                     right_on = ['symbol','program_id'])



In [374]:
combined_df

,PLANID,PLANINVCLOSEDATE,FUNDID,partner_program_id,model_props_id,model_id,symbol,program_id
0,VW0008000039,active,Medicham,8.0,NaN,NaN,NaN,NaN
1,VW0008000039,active,Arcanine,8.0,311.0,65.0,Arcanine,8.0
2,VW0008000039,11/01/2018,Clefairy,8.0,297.0,65.0,Clefairy,8.0
3,VW0008000039,active,Zubat,8.0,304.0,65.0,Zubat,8.0
4,VW0008000039,active,Nidoking,8.0,296.0,65.0,Nidoking,8.0
5,VW0008000039,active,Jigglypuff,8.0,302.0,65.0,Jigglypuff,8.0
6,VW0008000039,active,CharizardMega Charizard X,8.0,299.0,65.0,CharizardMega Charizard X,8.0
7,VW0008000039,active,Electrike,8.0,NaN,NaN,NaN,NaN
8,VW0008000039,active,Growlithe,8.0,310.0,65.0,Growlithe,8.0
9,VW0008000039,active,Fearow,8.0,286.0,55.0,Fearow,8.0


### Drop Columns with "Medicham" or "Electrike"

In [375]:
dropindex = combined_df[combined_df['FUNDID'] == 'Medicham' ].index

In [376]:
#delete later
dropindex

Int64Index([0, 76, 91, 145, 223, 272, 305, 331, 347, 396, 443, 482], dtype='int64')

In [377]:
combined_df.drop(dropindex, inplace=True)

In [378]:
dropindex = combined_df[combined_df['FUNDID'] == 'Electrike' ].index

In [379]:
#delete later
dropindex

Int64Index([  7,  61,  88, 114, 131, 152, 185, 220, 229, 262, 295, 320, 342,
            372, 408, 430, 450, 500],
           dtype='int64')

In [380]:
combined_df.drop(dropindex, inplace=True)

### Make complete values in program_id column

In [381]:
def get_program_id_complete(dfx):
    if math.isnan(dfx['program_id']):
        return dfx['partner_program_id']
    else:
        return dfx['program_id']

In [382]:
combined_df.program_id = combined_df.apply(get_program_id_complete, axis = 1)

In [383]:
combined_df

,PLANID,PLANINVCLOSEDATE,FUNDID,partner_program_id,model_props_id,model_id,symbol,program_id
1,VW0008000039,active,Arcanine,8.0,311.0,65.0,Arcanine,8.0
2,VW0008000039,11/01/2018,Clefairy,8.0,297.0,65.0,Clefairy,8.0
3,VW0008000039,active,Zubat,8.0,304.0,65.0,Zubat,8.0
4,VW0008000039,active,Nidoking,8.0,296.0,65.0,Nidoking,8.0
5,VW0008000039,active,Jigglypuff,8.0,302.0,65.0,Jigglypuff,8.0
6,VW0008000039,active,CharizardMega Charizard X,8.0,299.0,65.0,CharizardMega Charizard X,8.0
8,VW0008000039,active,Growlithe,8.0,310.0,65.0,Growlithe,8.0
9,VW0008000039,active,Fearow,8.0,286.0,55.0,Fearow,8.0
10,VW0008000039,active,Ekans,8.0,287.0,56.0,Ekans,8.0
11,VW0008000039,active,Arbok,8.0,288.0,57.0,Arbok,8.0


### Confirm no nan in program_id

In [384]:
combined_df.isna().sum()

PLANID                276
PLANINVCLOSEDATE      276
FUNDID                276
partner_program_id    276
model_props_id         26
model_id               26
symbol                 26
program_id              0
dtype: int64

In [385]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 755 entries, 1 to 784
Data columns (total 8 columns):
PLANID                479 non-null object
PLANINVCLOSEDATE      479 non-null object
FUNDID                479 non-null object
partner_program_id    479 non-null float64
model_props_id        729 non-null float64
model_id              729 non-null float64
symbol                729 non-null object
program_id            755 non-null float64
dtypes: float64(4), object(4)
memory usage: 53.1+ KB


In [386]:
#delete later
combined_df.FUNDID.value_counts()

Diglett                      20
CharizardMega Charizard X     9
Charmander                    9
Slowbro                       7
Rapidash                      7
Farfetch'd                    7
Magnemite                     7
BlastoiseMega Blastoise       6
Gastly                        6
CharizardMega Charizard Y     6
Charmeleon                    6
Bulbasaur                     6
SlowbroMega Slowbro           5
Doduo                         5
Slowpoke                      5
Blastoise                     5
Pikachu                       5
VenusaurMega Venusaur         5
Magneton                      5
Ponyta                        5
Charizard                     4
Raticate                      4
Sandshrew                     4
Nidorina                      4
Sandslash                     4
Growlithe                     4
Spearow                       4
Venusaur                      4
Raichu                        4
Magikarp                      4
                             ..
Staryu  

### Make symbol and FUNDID cells into lists (WE MAY NOT NEED THIS)

In [341]:
# combined_df.FUNDID = combined_df.FUNDID.apply(lambda x: str(x).split())
# combined_df.symbol = combined_df.symbol.apply(lambda x: str(x).split())

In [387]:
combined_df

,PLANID,PLANINVCLOSEDATE,FUNDID,partner_program_id,model_props_id,model_id,symbol,program_id
1,VW0008000039,active,Arcanine,8.0,311.0,65.0,Arcanine,8.0
2,VW0008000039,11/01/2018,Clefairy,8.0,297.0,65.0,Clefairy,8.0
3,VW0008000039,active,Zubat,8.0,304.0,65.0,Zubat,8.0
4,VW0008000039,active,Nidoking,8.0,296.0,65.0,Nidoking,8.0
5,VW0008000039,active,Jigglypuff,8.0,302.0,65.0,Jigglypuff,8.0
6,VW0008000039,active,CharizardMega Charizard X,8.0,299.0,65.0,CharizardMega Charizard X,8.0
8,VW0008000039,active,Growlithe,8.0,310.0,65.0,Growlithe,8.0
9,VW0008000039,active,Fearow,8.0,286.0,55.0,Fearow,8.0
10,VW0008000039,active,Ekans,8.0,287.0,56.0,Ekans,8.0
11,VW0008000039,active,Arbok,8.0,288.0,57.0,Arbok,8.0


### Merge is somehow fucked

In [400]:
combined_df[combined_df.program_id==1]

,PLANID,PLANINVCLOSEDATE,FUNDID,partner_program_id,model_props_id,model_id,symbol,program_id
483,VWPALL000076,active,Hitmontop,1.0,1906.0,269.0,Hitmontop,1.0
484,VWPALL000076,active,Smoochum,1.0,1910.0,269.0,Smoochum,1.0
485,VWPALL000076,active,Diglett,1.0,1848.0,261.0,Diglett,1.0
486,VWPALL000076,active,Diglett,1.0,1856.0,262.0,Diglett,1.0
487,VWPALL000076,active,Diglett,1.0,1864.0,263.0,Diglett,1.0
488,VWPALL000076,active,Diglett,1.0,1907.0,269.0,Diglett,1.0
489,VWPALL000076,active,Diglett,1.0,1878.0,265.0,Diglett,1.0
490,VWPALL000076,active,Diglett,1.0,1885.0,266.0,Diglett,1.0
491,VWPALL000076,active,Diglett,1.0,1891.0,267.0,Diglett,1.0
492,VWPALL000076,active,Diglett,1.0,1897.0,268.0,Diglett,1.0


### Group combined_df for analysis

In [401]:
combined_df.groupby(['program_id','FUNDID','symbol','PLANINVCLOSEDATE'])[['model_id']].sum()

model_id
program_id FUNDID                    symbol                    PLANINVCLOSEDATE          
1.0        Diglett                   Diglett                   active              2645.0
           Gastly                    Gastly                    active               269.0
           Growlithe                 Growlithe                 active               269.0
           Hitmontop                 Hitmontop                 active               269.0
           Meowth                    Meowth                    active               269.0
           Smeargle                  Smeargle                  active               269.0
           Smoochum                  Smoochum                  active               269.0
           Sneasel                   Sneasel                   active               269.0
           Stantler                  Stantler                  active               269.0
           Teddiursa                 Teddiursa                 active               269.0
           Torchic                   Torchic                   active               260.0
           Tyrogue                   Tyrogue                   active               269.0
           Xatu                      Xatu                      active               269.0
8.0        Arbok                     Arbok                     active                57.0
           Arcanine                  Arcanine                  active                65.0
           CharizardMega Charizard X CharizardMega Charizard X active                65.0
           Clefable                  Clefable                  active                65.0
           Clefairy                  Clefairy                  11/01/2018            65.0
           Ekans                     Ekans                     active                56.0
           Fearow                    Fearow                    active                55.0
           Gloom                     Gloom                     active                65.0
           Golbat                    Golbat                    active                65.0
           Growlithe                 Growlithe                 active                65.0
           Jigglypuff                Jigglypuff                active                65.0
           Nidoking                  Nidoking                  active                65.0
           Nidoqueen                 Nidoqueen                 active                63.0
           Nidorina                  Nidorina                  active                62.0
           Nidorino                  Nidorino                  active                64.0
           Ninetales                 Ninetales                 active                65.0
           Oddish                    Oddish                    active                65.0
...                                                                                   ...
29.0       Smeargle                  Smeargle                  active               346.0
           Smoochum                  Smoochum                  active               346.0
           Sneasel                   Sneasel                   active               346.0
           Stantler                  Stantler                  active               346.0
           Teddiursa                 Teddiursa                 active               346.0
           Tyrogue                   Tyrogue                   active               346.0
           Xatu                      Xatu                      active               346.0
30.0       Arbok                     Arbok                     active               765.0
           Charmander                Charmander                active               387.0
           Ekans                     Ekans                     active               764.0
           Fearow                    Fearow                    active               763.0
           Grovyle                   Grovyle                   active               387.0
           Growlithe           

In [392]:
combined_df.groupby(['program_id','FUNDID','symbol','PLANINVCLOSEDATE'])[['model_id']].sum().unstack()

model_id  \
PLANINVCLOSEDATE                                                 active   
program_id FUNDID                    symbol                               
1.0        Diglett                   Diglett                     2645.0   
           Gastly                    Gastly                       269.0   
           Growlithe                 Growlithe                    269.0   
           Hitmontop                 Hitmontop                    269.0   
           Meowth                    Meowth                       269.0   
           Smeargle                  Smeargle                     269.0   
           Smoochum                  Smoochum                     269.0   
           Sneasel                   Sneasel                      269.0   
           Stantler                  Stantler                     269.0   
           Teddiursa                 Teddiursa                    269.0   
           Torchic                   Torchic                      260.0   
           Tyrogue                   Tyrogue                      269.0   
           Xatu                      Xatu                         269.0   
8.0        Growlithe                 Growlithe                     65.0   
           Arbok                     Arbok                         57.0   
           Arcanine                  Arcanine                      65.0   
           CharizardMega Charizard X CharizardMega Charizard X     65.0   
           Clefable                  Clefable                      65.0   
           Clefairy                  Clefairy                       NaN   
           Ekans                     Ekans                         56.0   
           Fearow                    Fearow                        55.0   
           Gloom                     Gloom                         65.0   
           Golbat                    Golbat                        65.0   
           Jigglypuff                Jigglypuff                    65.0   
           Nidoking                  Nidoking                      65.0   
           Nidoqueen                 Nidoqueen                     63.0   
           Nidorina                  Nidorina                      62.0   
           Nidorino                  Nidorino                      64.0   
           Ninetales                 Ninetales                     65.0   
           Oddish                    Oddish                        65.0   
...                                                                 ...   
29.0       Smeargle                  Smeargle                     346.0   
           Smoochum                  Smoochum                     346.0   
           Sneasel                   Sneasel                      346.0   
           Stantler                  Stantler                     346.0   
           Teddiursa                 Teddiursa                    346.0   
           Tyrogue                   Tyrogue                      346.0   
           Xatu                      Xatu                         346.0   
30.0       Growlithe                 Growlithe                    387.0   
           Arbok                     Arbok                        765.0   
           Ekans                     Ekans                        764.0   
           Fearow                    Fearow                       763.0   
           Jigglypuff                Jigglypuff                   387.0   
           Nidoqueen                 Nidoqueen                    772.0   
           Nidorina                  Nidorina                     771.0   
           Nidorino                  Nidorino                     773.0   
           Pikachu                   Pikachu                     1146.0   
           Raichu                    Raichu                       768.0   
           Raticate                  Raticate                     761.0   
           Sandshrew                 Sandshrew                    769.0   
           Sandslash                 Sandslash                    770.0   
           Sp

### Now make ouput_df 

In [196]:
output_df = pd.DataFrame(columns=['program_id', 'fund_missing_at_vw', 'fund_missing_at_partner', 'fund_closed'])

In [197]:
output_df

,program_id,fund_missing_at_vw,fund_missing_at_partner,fund_closed


In [137]:
model_prop.groupby(["model_id",'model_props_id',"symbol"]).size().reset_index().rename(columns={0:'count'})

,model_id,model_props_id,symbol,count
0,17,107,Caterpie,1
1,18,108,Metapod,1
2,19,109,Butterfree,1
3,20,110,Weedle,1
4,21,111,Kakuna,1
5,22,112,Beedrill,1
6,23,113,BeedrillMega Beedrill,1
7,24,114,Pidgey,1
8,28,119,Caterpie,1
9,28,120,Metapod,1


In [141]:
model_prop[model_prop.model_id==28]

,model_props_id,model_id,symbol
13,119,28,Caterpie
14,120,28,Metapod
15,121,28,Butterfree
16,122,28,Weedle
17,123,28,Kakuna
26,124,28,Beedrill
27,125,28,BeedrillMega Beedrill
28,126,28,Pidgey
29,127,28,Pidgeotto
30,128,28,Pidgeot


In [136]:
# model_prop.groupby(["model_id",'model_props_id']).describe()

symbol                                   
                         count unique                    top freq
model_id model_props_id                                          
17       107                 1      1               Caterpie    1
18       108                 1      1                Metapod    1
19       109                 1      1             Butterfree    1
20       110                 1      1                 Weedle    1
21       111                 1      1                 Kakuna    1
22       112                 1      1               Beedrill    1
23       113                 1      1  BeedrillMega Beedrill    1
24       114                 1      1                 Pidgey    1
28       119                 1      1               Caterpie    1
         120                 1      1                Metapod    1
         121                 1      1             Butterfree    1
         122                 1      1                 Weedle    1
         123                 1      1                 Kakuna    1
         124                 1      1               Beedrill    1
         125                 1      1  BeedrillMega Beedrill    1
         126                 1      1                 Pidgey    1
         127                 1      1              Pidgeotto    1
         128                 1      1                Pidgeot    1
         129                 1      1    PidgeotMega Pidgeot    1
29       157                 1      1                Rattata    1
         158                 1      1                  Paras    1
         159                 1      1               Parasect    1
         160                 1      1                Venonat    1
         161                 1      1               Venomoth    1
30       162                 1      1                  Paras    1
         163                 1      1               Parasect    1
         164                 1      1                Venonat    1
         165                 1      1               Venomoth    1
         190                 1      1                Diglett    1
         192                 1      1                Rattata    1
...                        ...    ...                    ...  ...
380      16936               1      1                Pikachu    1
381      2918                1      1                 Raichu    1
382      2919                1      1              Sandshrew    1
383      2920                1      1              Sandslash    1
384      2921                1      1               Nidorina    1
385      2922                1      1              Nidoqueen    1
386      2923                1      1               Nidorino    1
387      2924                1      1              Growlithe    1
         2925                1      1             Charmander    1
         2926                1      1                 Wooper    1
         2927                1      1                  Yanma    1
         2928                1      1                Treecko    1
         2929                1      1                Grovyle    1
         2930                1      1             Jigglypuff    1
         2931                1      1                Haunter    1
         2932                1      1             Wigglytuff    1
         2933                1      1               Venusaur    1
         2934                1      1               Sceptile    1
         2935                1      1               Raticate    1
         2936                1      1                Spearow    1
         2937                1      1                 Fearow    1
         2938                1      1                  Ekans    1
         2939                1      1                  Arbok    1
         2940                1      1                Pikachu    1
         2941                1      1                 Raichu    1
         2942                1      1              Sandshrew    1
         2943                1      1              Sandslash    

In [124]:
model.join(model_prop, on='model_id')

ValueError: columns overlap but no suffix specified: Index(['model_id'], dtype='object')

# Step 4 - Check for any closed funds
Check each `program_id` to see if our partner has indicated a fund that is in Vestwell's `model` has been closed and add that to the output from Step 3.